#### Instruction: put in your model name and run cells sequentially

In [ ]:
from parse_entities import *
import pandas as pd
model = '[Your GPT model here]'

In [ ]:
def make_batch(text_list:List[str], identifier_list:List[str],store_directory:str)-> None: 
    system_prompt = "You are an expert in labeling Personally Identifiable Information. Start your response rightaway without adding any prefix(such as Response:) and suffix"
    ins_prompt = '''Label the entity of the following text: use @@@,### to label student name\n
Ensure that the rest of the text remains unchanged, word for word.
Maintain the original punctuation, quotation marks, spaces, and line breaks. 
If the text does not contain any PII, return it as is.
For example, if the input is:
student: Hi\nstudent: Boria Lupu?\nteacher: Hi Gurung Kool!\nteacher: How are you today?\nstudent: Fine\n
The output should be:
student: Hi\nstudent: @@@Boria Lupu###?\nteacher: Hi @@@Gurung Kool###!\nteacher: How are you today?\nstudent: Fine\n
Another example:
teacher: I presume 'agape' is Greek\nstudent: I think so. It's a very rich language\nstudent: I can ask our teacher Josue Rodrigo Nieto Hernandez. He knows Greek\n
The output should be:
teacher: I presume 'agape' is Greek\nstudent: I think so. It's a very rich language\nstudent: I can ask our teacher @@@Josue Rodrigo Nieto Hernandez###. He knows Greek\n
Another example:
student: Yesterday Okpan Ohabuike's mum had a surgery you know\nteacher: Oh yes, of course! Is she ok?
The output should be exactly the same as input:
student: Yesterday @@@Okpan Ohabuike###'s mum had a surgery you know\nteacher: Oh yes, of course! Is she ok?
Please repeat this process with the following file:\n
'''
    # model="gpt-4o-mini-2024-07-18"
    model= model
    try:
        os.makedirs(store_directory, exist_ok=True)
        print(f"Directory '{store_directory}' is ready.")
    except Exception as e:
        print(f"Error creating directory '{store_directory}': {e}")
    res = []
    for i in range(len(text_list)): 
        request = {"custom_id": f'{identifier_list[i]}', "method": "POST", 
                  "url": "/v1/chat/completions", 
                  "body": {"model": model, "messages": [{"role": "system", "content": system_prompt},{"role": "user", "content": ins_prompt + text_list[i]}],
                  "temperature":0}}
        res.append(request)
    
    store_directory += '/batch_request.jsonl'
    with open(store_directory, 'w',encoding='utf-8') as f:
        for entry in res:
            json_line = json.dumps(entry, ensure_ascii=False)
            f.write(json_line + '\n')

In [ ]:
with open('output/new_texts.txt', 'r') as f:
    input_texts = json.load(f)

In [ ]:
# Provide the directory to store the temporary files
gpt_storage_directory = "gpt_output_ft+pt"
identifier_list = [f'doc{i}' for i in range(len(input_texts))]

In [ ]:
#make batch file for submitting finetune job 
make_batch(input_texts, identifier_list,gpt_storage_directory)

#submit finetune job 
submit_finetune_job(gpt_storage_directory)

#print the progress
check_progress(gpt_storage_directory)

In [ ]:
check_progress(gpt_storage_directory)

In [ ]:
#check progress, if completed, then retrieve the file_id
out_id = check_progress(gpt_storage_directory)
if out_id != None: 
    #if comleted, retrieve the result and store the result in a file.
    file_response = client.files.content(out_id)
    save_file_response_as_jsonl(file_response, gpt_storage_directory)  
    data_list = read_jsonl_file(gpt_storage_directory)
    data = resort_data(data_list,identifier_list)
    batch_extract_parse_store(data,identifier_list,gpt_storage_directory)

In [ ]:
detected_entities = extract_entities(r'gpt_output_prompting/gpt_result.json')
df = pd.DataFrame(detected_entities, columns=['file_idx', 'entity_text', 'positions'])

In [ ]:
output_file = 'results/TSCC_detected_prompting.csv'
df.to_csv(output_file, index=False)

print(f"CSV file saved as {output_file}")